In [1]:
import jax
import jax.numpy as jnp
import jax.scipy as jsp
from jax import grad, jit, vmap
from jax import random
from jax import jacfwd
import jax_cosmo as jc
import numpy
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.stats import exponnorm
import sncosmo
from astropy.table import Table
import pandas
import pickle

In [2]:
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platform_name", "cpu")
jax.config.update("jax_debug_nans", True)

# Calculate the Fisher Matrix of SN programs

The data are m and measurement uncertainty dm

The model parameters are

Omega: Cosmological
c: calibration
Av, Rv: per supernova dust

The likelihood p(y| Omega, c, Av, Rv) is
N(c,dc) P(Av, Rv) Prod_SN N(m-(M+mu+C(c)+D(Av, Rv), dm_I^2 + dm^2)

The terms are
<partial ^2 lnL / partial alpha partial beta> = -partial X / partial alpha C^{-1} partial X /beta + delta_alpha,beta d2(ln p_alpha)


In [3]:
## The SN Model

In [4]:
class SNModel(object):
    """
    Class for a SN Ia SED at maximum integrated by a top-hat resolution R.
    The spectrum is the mean from SALT2

    Attributes
    ----------
        R (float): The resolution of the convolution kernel
    """
    
    def __init__(self):
        """
        Constructor for SN model
     
        Parameters
        ----------
        R : float
            Resolution that the SED is convolved
     
        Returns
        -------
        float
            The quotient of the division.
        """
        
        phase, self.wave, values = sncosmo.io.read_griddata_ascii('../data/salt2-k21-frag/salt2_template_0.dat')
        # buffer the arrays with zeros to ensure linear interpolation behaves OK
        self.wave = jnp.concatenate((numpy.array([1000]), self.wave, numpy.array([20000])))
        self.values=jnp.concatenate((numpy.array([0]),values[numpy.where(phase==0)[0][0],:],numpy.array([0])))
        # self.R = R
        
    @staticmethod
    def interp_(x, xp, fp):
        """
        Simple equivalent of np.interp that compute a linear interpolation.
    
        We are not doing any checks, so make sure your query points are lying
        inside the array.
    
        Copied from jax_cosmo
    
        Parameters
        ----------
        x : float
            Where to interpolate
        xp: array
            Input x-axis need to be 1d arrays
        fp: array
            Input y axis need to be 1d arrays
     
        Returns
        --------
        float: interpolated flux value
        """
        # First we find the nearest neighbour
        ind = jnp.argmin((x - xp) ** 2)
    
        # Perform linear interpolation
        ind = jnp.clip(ind, 1, len(xp) - 2)
        xi = jnp.asarray(xp)[ind]
    
        # Figure out if we are on the right or the left of nearest
        s = jnp.sign(jnp.clip(x, jnp.asarray(xp)[1], jnp.asarray(xp)[-2]) - xi).astype(np.int64)
        a = (jnp.asarray(fp)[ind + jnp.copysign(1, s).astype(np.int64)] - jnp.asarray(fp)[ind]) / (
            jnp.asarray(xp)[ind + jnp.copysign(1, s).astype(np.int64)] - jnp.asarray(xp)[ind]
        )
        b = jnp.asarray(fp)[ind] - a * jnp.asarray(xp)[ind]
        
        return a * x + b
        
    @staticmethod
    def interp(x,xp,fp):
        """
        Interpolation for an array of x values
    
        Parameters
        ----------
        x : array
            Where to interpolate
        xp: array
            Input x-axis need to be 1d arrays
        fp: array
            Input y axis need to be 1d arrays
        
        Returns
        --------
        array: interpolated flux values
        """
        return vmap(interp_,(0, None, None))(x,xp,fp)

    
    # def flux(self,wave_in):
    #     """
    #     Returns the native (unconvolved) flux value at input wavelength. 
    
    #     Parameters
    #     ----------
    #     wave_in : array
    #         Where to interpolate


    #     """
    #     return interp(wave_in,self.wave,self.values)

    # def bandmag_one(self,wl, moments=False):
    #     """
    #     Returns the integrated flux magnitude value at input wavelength.  Optionally calculated
    #     first several moments of the 
    
    #     Parameters
    #     ----------
    #     wl : float
    #         Where to interpolate
    #     moments : boolean (default=False)
    #         Calculate the moments

    #     Returns
    #     -------
    #     float: magnitude of the filter (if default=False)
    #     float, float, float, float: magnitude of the filter, mean, variance, and skewness (if default=True)
    #     """
    #     # determine edges of the filter
    #     delta = numpy.arcsinh(1/2/self.R)
    #     lmin= wl*numpy.exp(-delta)
    #     lmax= wl*numpy.exp(delta)
        
    #     # determine the native fluxes at the filter edges
    #     vall = SNModel.interp_(lmin,self.wave,self.values)
    #     valr = SNModel.interp_(lmax,self.wave,self.values)

    #     # zero out the fluxes not covered by the filter, add the edge points, to create
    #     # arrays to integrate
    #     logic = jnp.logical_and(self.wave > lmin, self.wave < lmax)
    #     _flux = self.values * logic
    #     _wave  = jnp.concatenate((jnp.array([lmin]), self.wave, jnp.array([lmax])))
    #     _flux = jnp.concatenate((jnp.array([vall]), _flux, jnp.array([valr])))
    #     args = jnp.argsort(_wave)
    #     _wave=_wave[args]
    #     _flux=_flux[args]
        
    #     # perform the integration
    #     integrated_flux = jax.scipy.integrate.trapezoid(_flux,_wave)
    #     if moments:
    #         mean = jax.scipy.integrate.trapezoid(jnp.log(_wave)*_flux,_wave)/integrated_flux
    #         variance = jax.scipy.integrate.trapezoid((jnp.log(_wave)-mean)**2*_flux,_wave)/integrated_flux
    #         skewness = jax.scipy.integrate.trapezoid((jnp.log(_wave)-mean)**3*_flux,_wave)/integrated_flux/variance**1.5
    #         return -2.5*jnp.log10(integrated_flux), mean, variance, skewness
    #     else:
    #         return -2.5*jnp.log10(integrated_flux)

In [5]:
## Dust Model

In [6]:
@jit
def CCM_a(wl):
    """
    CCM a function as a function of wavelength
 
    Parameters
    ----------
    wl : float
        Resolution that the SED is convolved
 
    Returns
    -------
    float
        Value of the function.
    """
    def infrared(x):
        a = 0.574*x**1.61
        return a

    def optical(x):
        y=x-1.82
        a = 1 + 0.17699*y - 0.50447*y**2 - 0.02427*y**3 + 0.72085*y**4+ 0.01979*y**5 - 0.77530*y**6 + 0.32999*y**7
        return a

    def ultraviolet(x):
        a = 1.752 - 0.316*x - 0.104/((x - 4.67)**2 + 0.341)
        return a
        
    x = 1e4/wl
    return jnp.select(condlist=[(x >= 0.3) & (x<=1.1), (x >= 1.1) & (x<=3.3), (x >= 3.3) & (x<=5.9)],
                                choicelist=[infrared(x), optical(x), ultraviolet(x)], default=0.)
def CCM_a_multi(wl):
    return vmap(CCM_a,(0))(wl)

@jit
def CCM_b(wl):
    """
    CCM b function as a function of wavelength
 
    Parameters
    ----------
    wl : float
        Resolution that the SED is convolved
 
    Returns
    -------
    float
        Value of the function.
    """
    def infrared(x):
        b = -0.527*x**1.61
        return b

    def optical(x):
        y=x-1.82
        b = 1.41338*y + 2.28305*y**2 + 1.07233*y**3 - 5.38434*y**4 - 0.62251*y**5 + 5.30260*y**6 - 2.09002*y**7
        return b

    def ultraviolet(x):
        b = -3.090 + 1.825*x + 1.206/((x - 4.62)**2 + 0.263)
        return b
        
    x = 1e4/wl
    return jnp.select(condlist=[(x >= 0.3) & (x<=1.1), (x >= 1.1) & (x<=3.3), (x >= 3.3) & (x<=5.9)],
                                choicelist=[infrared(x), optical(x), ultraviolet(x)], default=0.)

def CCM_b_multi(wl):
    return vmap(CCM_b,(0))(wl)

In [7]:
# distribution of Av Exponentially modified Gaussian distribution

# Reasonable values in wikipedia notation
Avmu = 0.
Avlambda = 5.
Avsigma = 0.15

# in scipy
AvK = 1./Avlambda/Avsigma
Avscale = Avsigma

def pAvrvs(size=1):
    return  exponnorm.rvs(size=size, K=AvK, loc=0, scale=Avscale)
    
def d2lnpAv(x):
    # d2 ln efrc((mu + lambda sigma^2 - x)/sqrt2/sigma)
    # z = (mu + lambda sigma^2 - x)/sqrt2/sigma
    # d1 = 1/erfc(z) (-2/sqrt(pi) exp(-z^2) (-1/sqrt2/sigma)
    # d2 = 
    z = (Avmu + Avlambda*Avsigma*Avsigma - x)/numpy.sqrt(2)/Avsigma
    erfc_ = scipy.special.erfc(z)
    exp_ = numpy.exp(-z**2)
    return 4*exp_/numpy.sqrt(numpy.pi)/Avscale/Avscale/erfc_ * (z - exp_/numpy.sqrt(numpy.pi)/erfc_)

# distribution of Rv Gaussian distribution

def d2lnpRv(x):
    sigma=0.3
    ans = numpy.zeros(len(x))
    ans.fill(-1./sigma/sigma)

    return ans

## Source plus dust information

In [8]:
def bandmag_one(snmodel, Res, Av, Rv, wl, moments=False):
    """
    Returns the integrated flux magnitude value at input wavelength.  Optionally calculated
    first several moments of the 

    Parameters
    ----------
    wl : float
        Where to interpolate
    moments : boolean (default=False)
        Calculate the moments

    Returns
    -------
    float: magnitude of the filter (if default=False)
    float, float, float, float: magnitude of the filter, mean, variance, and skewness (if default=True)
    """
    # determine edges of the filter
    delta = numpy.arcsinh(1/2/Res)
    lmin= wl*numpy.exp(-delta)
    lmax= wl*numpy.exp(delta)
    
    # determine the native fluxes at the filter edges
    vall = SNModel.interp_(lmin,snmodel.wave,snmodel.values)
    valr = SNModel.interp_(lmax,snmodel.wave,snmodel.values)

    # zero out the fluxes not covered by the filter, add the edge points, to create
    # arrays to integrate
    logic = jnp.logical_and(snmodel.wave > lmin, snmodel.wave < lmax)
    _flux = snmodel.values * logic
    _wave  = jnp.concatenate((jnp.array([lmin]), snmodel.wave, jnp.array([lmax])))
    _flux = jnp.concatenate((jnp.array([vall]), _flux, jnp.array([valr]))) * 10**(-(Av*(CCM_a_multi(_wave)+CCM_b_multi(_wave)/Rv))/2.5)
    args = jnp.argsort(_wave)
    _wave=_wave[args]
    _flux=_flux[args]
    
    # perform the integration
    integrated_flux = jax.scipy.integrate.trapezoid(_flux,_wave)
    if moments:
        mean = jax.scipy.integrate.trapezoid(jnp.log(_wave)*_flux,_wave)/integrated_flux
        variance = jax.scipy.integrate.trapezoid((jnp.log(_wave)-mean)**2*_flux,_wave)/integrated_flux
        skewness = jax.scipy.integrate.trapezoid((jnp.log(_wave)-mean)**3*_flux,_wave)/integrated_flux/variance**1.5
        return -2.5*jnp.log10(integrated_flux), mean, variance, skewness
    else:
        return -2.5*jnp.log10(integrated_flux)

In [9]:
#deprecated
# def color(wl, Av, Rv):
#     DeltaLnLambda=0.26  # rough separation between filters 
#     # in dm/dlnl
#     # return grad(lambda x: M_sn(x)+Av*(CCM_a(x)+CCM_b(x)/Rv), (0))(wl)/wl
#     return grad(lambda x: sn.bandmag_one(x)+Av*(CCM_a(x)+CCM_b(x)/Rv), (0))(wl)*wl*DeltaLnLambda

# def color_run():
#     print (color(4400., 0.1, 3.1))

## Chi-sq constribution to Fisher Matrix (not including dm_int term which is another part of the code)

In [10]:
# For one object one band
def mbar_perfilt(Omega_c, w0, wa, dC, Av, Rv, z, efflam, snmodel, Res):
    """
    Expected magnitude in one filter for one supernova

    Parameters
    ----------
    Omega_c: float
    w0: float
    wa: float
    dC: array
        Color parameters whose effect on the magnitude encoded
    Av:
    Rv:
    z:
    efflam: float
        effective wavelength of the filter
    snmodel: SNModel
    R: float
        resolution of the filter

    Returns
    -------
    float: magnitude of the filter
    """
    # mu not efficient
    cosmo = jc.Planck15(Omega_c=Omega_c, w0=w0, wa=wa)

    # convert to SN frame
    # include filter wavelength calibration error
    restlam=(efflam+dC[1])/(1+z)
 
    mu =  10*jnp.log10(1+z) + 5*jnp.log10(jc.background.angular_diameter_distance(cosmo, 1/(1+z)))[0]

    M, mean, variance, skewness = bandmag_one(snmodel, Res, Av, Rv, restlam, moments=True)
    
    # calibration terms shift the expected magnitude
    return M + mu + dC[0] + dC[2]*skewness

def dmbar_perfilt(Omega_c, w0, wa, dC, Av, Rv, z, efflam, snmodel, Res):
    """
    Derivative of the expected magnitude in one filter for one supernova

    Parameters
    ----------
    Omega_c: float
    w0: float
    wa: float
    dC: array
        Color parameters whose effect on the magnitude encoded
    Av:
    Rv:
    z:
    efflam: float
        effective wavelength of the filter
    snmodel: SNModel
    R: float
        resolution of the filter

    Returns
    -------
    float: magnitude of the filter
    """
    return grad(mbar_perfilt,(0,1,2,3,4,5))(Omega_c, w0, wa, dC, Av, Rv, z, efflam, snmodel, Res)

def dmbar(Omega_c, w0, wa, dCs, Av, Rv, z, efflams, snmodel, Res):
    """
    Derivative of the expected magnitude of multiple filters for one supernova

    Parameters
    ----------
    Omega_c: float
    w0: float
    wa: float
    dC: array
        Color parameters whose effect on the magnitude encoded
    Av:
    Rv:
    z:
    efflam: array
        effective wavelength of the filter
    snmodel: SNModel
    R: float
        resolution of the filters

    Returns
    -------
    array: magnitude of the filters
    """    
    return vmap(dmbar_perfilt, (None, None, None, 0,  None, None, None, 0, None, None))(Omega_c, w0, wa, dCs, Av, Rv, z, efflams, snmodel, Res)
    

# def mbar(Omega_c, w0, wa, dCs, Av, Rv, z, efflams, snmodel, R):
#     return vmap(mbar_perfilt, (None, None, None, 0,  None, None, None, 0, None, None))(Omega_c, w0, wa, dCs, Av, Rv, z, efflams, snmodel, R)

def mbar_check():
    nsn=10
    efflams = numpy.array([4700., 6420.,  7849.])
    zs = numpy.linspace(0.1,1,nsn)
    Avs = pAV().rvs(size=nsn, scale=AVscale)
    Rvs = numpy.zeros(nsn)+3.1
    dCs= numpy.zeros((3,3))
    Omega_c=0.3
    w0=-1.
    wa=0.
    # ans = mbar_perfilt(Omega_c, w0, wa,dCs[0],Avs[0],Rvs[0],zs[0],efflams[0])
    # dans = dmbar_perfilt(Omega_c, w0, wa ,dCs[0],Avs[0],Rvs[0],zs[0],efflams[0])
    # print(ans)
    # for _ in dans:
    #     print(_)
    ans = mbar(Omega_c, w0, wa,dCs,Avs[0],Rvs[0],zs[0],efflams)
    dans = dmbar(Omega_c, w0, wa ,dCs,Avs[0],Rvs[0],zs[0],efflams)
    # print(ans)
    for _ in dans:
        print(_)

def F_chisq(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_datas, snmodel, Res):
    """
    Chi-square contrbution to the Fisher Matrix

    Parameters
    ----------
    Omega_c: float
    w0: float
    wa: float
    dCs: array of array
        Color parameters whose effect on the magnitude encoded
    Avs:
    Rvs:
    zs:
    efflam: array
        effective wavelength of the filter
    snmodel: SNModel
    R: float
        resolution of the filters

    Returns
    -------
    Matrix: chi-sq contribution to Fisher Matrix
    """   
    nfilt = len(efflams)
    nsn = len(Avs)
    ncal = len(dCs[0])
    ncosmo = 3
    npars = 3 + ncal*nfilt + 2*nsn  # 3 cosmology, ncal * nfilt, Av, Rv
                       
    F=numpy.zeros((npars,npars))
    
    #container for internal calculation
    partial = numpy.zeros(nfilt)
    
    # loop over SNe
    snind=0
    for Av , Rv, z, dm_data in zip( Avs, Rvs, zs, dm_datas):
        # partial derivatives of magnitudes of this SN w.r.t. all the parameters
        dmbars = dmbar(Omega_c, w0, wa, dCs, Av, Rv, z, efflams, snmodel, Res)

        # covariance matrix of the observations factored
        cov = numpy.zeros((nfilt,nfilt))+dm_int**2
        diag = numpy.identity(nfilt) * dm_data**2
        cov = cov + diag
        # Despite the documentation, the example shows L.T L = cov
        L, lower = jsp.linalg.cho_factor(cov)
    
        #Cosmo
        for i in range(ncosmo):
            rhs = jsp.linalg.solve_triangular(L, dmbars[i],trans=1,lower=lower)
            F[i,i] += jnp.dot(rhs,rhs)
            #Cosmo
            for j in range(i+1,ncosmo):
                lhs = jsp.linalg.solve_triangular(L, dmbars[j],trans=1,lower=lower)
                F[i,j] += jnp.dot(lhs,rhs)
            # Calibrations
            for j in range(nfilt):
                for k in range(ncal):
                    partial[:]=0
                    partial[j] = dmbars[ncosmo][j][k]  # j-calibration parameter only has partial with jth magnitude
                    lhs = jsp.linalg.solve_triangular(L, partial,trans=1,lower=lower)
                    F[i,ncosmo+ncal*j+k] += jnp.dot(lhs,rhs)
            # Av
            lhs = jsp.linalg.solve_triangular(L, dmbars[ncosmo+1],trans=1,lower=lower)
            F[i,ncosmo+ncal*nfilt+snind] += jnp.dot(lhs,rhs)
            
            # Rv
            lhs = jsp.linalg.solve_triangular(L, dmbars[ncosmo+2],trans=1,lower=lower)
            F[i,ncosmo+ncal*nfilt+nsn+snind] += jnp.dot(lhs,rhs)
        
        #Calibrations
        for i in range(nfilt):
            for l in range(ncal):
                partial[:]=0
                partial[i] = dmbars[ncosmo][i][l]
                rhs = jsp.linalg.solve_triangular(L, partial,trans=1,lower=lower)
                F[ncosmo+ncal*i+l,ncosmo+ncal*i+l] += jnp.dot(rhs,rhs)
                
                # Calibrations
                for j in range(i,nfilt):
                    for k in range(ncal):
                        partial[:]=0
                        partial[j] = dmbars[ncosmo][j][k]
                        lhs = jsp.linalg.solve_triangular(L, partial,trans=1,lower=lower)
                        F[ncosmo+ncal*i+l,ncosmo+ncal*j+k] += jnp.dot(lhs,rhs)
                # Av
                lhs = jsp.linalg.solve_triangular(L, dmbars[ncosmo+1],trans=1,lower=lower)
                F[ncosmo+ncal*i+l,ncosmo+ncal*nfilt+snind] += jnp.dot(lhs,rhs)
                
                # Rv
                lhs = jsp.linalg.solve_triangular(L, dmbars[ncosmo+2],trans=1,lower=lower)
                F[ncosmo+ncal*i+l,ncosmo+ncal*nfilt+nsn+snind] += jnp.dot(lhs,rhs)
        

        #Av
        rhs = jsp.linalg.solve_triangular(L, dmbars[ncosmo+1],trans=1,lower=lower)
        F[ncosmo+ncal*nfilt+snind, ncosmo+ncal*nfilt+snind] += jnp.dot(rhs,rhs)

            #Rv
        lhs = jsp.linalg.solve_triangular(L, dmbars[ncosmo+2],trans=1,lower=lower)
        F[ncosmo+ncal*nfilt+snind, ncosmo+ncal*nfilt+nsn+snind] += jnp.dot(lhs,rhs)

        #Rv
        F[ncosmo+ncal*nfilt+nsn+snind,ncosmo+ncal*nfilt+nsn+snind] += jnp.dot(lhs,lhs)
        
        snind+=1

    for i in range(npars):
        for j in range(i+1,npars):
            F[j,i]=F[i,j]

    return F
        
def F_chisq_check():
    ncal=3
    nsn=10
    efflams = numpy.array([4700., 6420.,  7849.])
    zs = numpy.linspace(0.1,1,nsn)
    Avs = pAvrvs(size=nsn1)
    Rvs = numpy.zeros(nsn)+3.1
    dCs= numpy.zeros((3,ncal))
    dCs=numpy.array([[1.,2,3],[4,5,6],[6,7,8]])
    Omega_c=0.3
    w0=-1.
    wa=0.
    dm_int=0.1
    dm_data=0.02
    F = F_chisq(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data)
    print(F[3:12,3:12])

# F_chisq_check()

## Chi-sq term for dm_int (Not implemented correctly so intrinsic dispersion is not a free parameter yet)

In [11]:
def lnL_dm_int(dm_int, dm_data, nfilt):
    
    # For now assume all data have same incertainties
    cov = numpy.zeros((nfilt,nfilt))+dm_int**2
    diag = numpy.identity(nfilt) * dm_data**2
    cov = cov + diag
    # Despite the documentation, the example shows L.T L = cov
    L, lower = jsp.linalg.cho_factor(cov)    
    return -0.5/jnp.diagonal(L).prod()**2

def d2ln_dm_int(dm_int, dm_data, nfilt):
    return grad(grad(lnL_dm_int,(0)),(0))(dm_int,dm_data, nfilt)

def F_chisq_dm_int(efflams, dm_int, dm_data, nsn):
    nfilt = len(efflams)

## Log det term (currently not included as intrinsic dispersion not being fit

In [12]:
def lnL_logdet(dm_int, dm_datas, efflams):
    nfilt=len(efflams)
    # For now assume all data have same incertainties
    cov = numpy.zeros((nfilt,nfilt))
    for dm_data in dm_datas:
        cov[:]=dm_int**2
        diag = numpy.identity(nfilt) * dm_data**2
        cov = cov + diag
        # Despite the documentation, the example shows L.T L = cov
        L, lower = jsp.linalg.cho_factor(cov)
    
        # logdet term
        ans = - jnp.log(jnp.diagonal(L)).sum()
    return ans

def F_logdet(dm_int, dm_data, efflams):
    return grad(grad(lnL_logdet,(0)),(0))(dm_int, dm_data, efflams)

def F_logdet_check():
    dm_int=0.1
    dm_data=0.02
    efflams = numpy.array([4700., 6420.,  7849.])
    print(F_logdet(dm_int, dm_data, efflams))

## AV and RV terms

In [13]:
def F_Av(Avs):
    ans = []
    for Av in Avs:
        ans.append(-d2lnpAv(Av))
    return ans

def F_Av_test():
    nsn=10
    Avs = pAv().rvs(size=nsn, scale=Avscale)    
    print(Avs, F_Av(Avs))

def F_Rv(Rvs):
    return -d2lnpRv(Rvs)
# F_Av_test()

## Combining all contributions to F (for now leaving out dm_int)

In [14]:
def F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data, snmodel, Res):
    """
    Fisher Matrix for a single survey.  Includes chi-sq, Av distribution, Rv distribution

    Calibration priors not included

    Parameters
    ----------
    Omega_c: float
    w0: float
    wa: float
    dCs: array of array
        Color parameters whose effect on the magnitude encoded
    Avs:
    Rvs:
    zs:
    efflam: array
        effective wavelength of the filter
    snmodel: SNModel
    R: float
        resolution of the filters

    Returns
    -------
    Matrix: Fisher Matrix
    """
    ncosmo =3
    nfilt = len(efflams)
    ncal=len(dCs[0])
    nsn = len(Avs)

    # chisq term fills everything
    F = F_chisq(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data, snmodel, Res)

    # add contributions from AV and RV distriutions
    Favs = F_Av(Avs)
    for i in range(nsn):
        F[ncosmo+ncal*nfilt+i, ncosmo+ncal*nfilt+i]+=Favs[i]
        
    Frvs = F_Rv(Rvs)
    for i in range(nsn):
        F[ncosmo+ncal*nfilt+nsn+i, ncosmo+ncal*nfilt+nsn+i]+=Frvs[i]  

    return F

## calibration uncertainty priors for a single Fisher Matrix

In [15]:
def addCalibrationUncertaintyPerF(ncosmo, nfilt, ncal,F):
    """
    Chi-square contrbution to the Fisher Matrix

    Parameters
    ----------
    ncosmo: int
        number of cosmological parameters
    nfilt: int
        number of filters
    ncal: int
        number of calibration parameters
    F:    int
        Fisher matrix to be modified
    """
    # I checked the ordering of the C variables
    ncalinvsig2 = [1/0.015/0.015, 1./64, 1/0.015/0.015]
    for i in range(nfilt):
        for j in range(ncal):
            F[ncosmo+i*nfilt+j,ncosmo+i*nfilt+j] += ncalinvsig2[j]

In [16]:
def addCalibrationUncertainty(ncosmo, nfilts, ncals, nsne, F, ncalinvsig2 = [1/0.015/0.015, 1./64, 1/0.015/0.015]):
    """
    Chi-square contrbution to the Fisher Matrix

    Parameters
    ----------
    ncosmo: int
        number of cosmological parameters
    nfilt: int
        number of filters
    ncal: int
        number of calibration parameters
    F:    int
        Fisher matrix to be modified
    """
    # I checked the ordering of the C variables
    for isurvey, (nfilt, ncal, nsn) in enumerate(zip(nfilts,ncals,nsne)):
        for i in range(nfilt):
            for j in range(ncal):
                if isurvey !=0:
                    index = ncosmo+(numpy.array(nfilts[:isurvey])*numpy.array(ncals[:isurvey])).sum() + 2 * numpy.sum(nsne[:isurvey]) +i *nfilt+j
                else:
                    index = ncosmo +i *nfilt+j
                F[index,index] += ncalinvsig2[j]

In [17]:
def F_merge(ncosmo, F1, F2):
    """
    Create a one Fisher Matrix from two.  The cosmological parameters are in common, the rest are not.


    Parameters
    ----------
    F1: matrix
        Fisher Matrix 1
    F2: matrix
        Fisher Matrix 2
        
    Returns
    -------
    Matrix: Merged Fisher Matrix
    """
    dim1 = F1.shape[0]
    dim2 = F2.shape[0]
    npars = dim1 + dim2 - ncosmo

    F = numpy.zeros((npars,npars))
    F[0:ncosmo, 0:ncosmo] = F1[0:ncosmo, 0:ncosmo]+ F2[0:ncosmo, 0:ncosmo]
    F[0:ncosmo, ncosmo:dim1] = F1[0:ncosmo,ncosmo:]
    F[0:ncosmo, dim1:] = F2[0:ncosmo,ncosmo:]
    F[ncosmo:dim1,0:dim1] = F1[ncosmo:,:]
    F[dim1:,0:ncosmo] = F2[ncosmo:,0:ncosmo]
    F[dim1:, dim1:] = F2[ncosmo:,ncosmo:]
    return F

In [18]:
def sqrtInvFisherDiag(F, ind):
    """
    Return the square root of the ind'th diagona of the inverse of F


    Parameters
    ----------
    F: matrix
        Fisher Matrix
    ind: int
        diagonal index of interest
        
    Returns
    -------
    float: square root of the ind'th diagona of the inverse of F
    """
    pick = numpy.zeros(F.shape[0])
    pick[ind]=1.
    L,lower = scipy.linalg.cho_factor(F)
    dum = jsp.linalg.solve_triangular(L, pick,trans=1,lower=lower)
    return numpy.linalg.norm(dum)

In [19]:
def Ftosigma(F, ncosmo, ncal, nfilt1, nsn1, nfilt2, nsn2, nfilt3, nsn3, nfilt4, nsn4):
    # F = numpy.array(F_hold)
    ans=numpy.zeros((2,4))    

    # indeces to delete 
    ind0=[]    # keep all
    ind1=[]    # keep zero and shift
    ind2=[]    # keep zero
    ind3=[]    # remove all calibration

    # remove indeces and Av, Rv
    for i in range(nfilt1):
        ind1.append(ncosmo+i*ncal+2)
        ind2.append(ncosmo+i*ncal+1)
        ind2.append(ncosmo+i*ncal+2)
        ind3.append(ncosmo+i*ncal)        
        ind3.append(ncosmo+i*ncal+1)
        ind3.append(ncosmo+i*ncal+2)        
    for j in range(nsn1):
        ind1.append(ncosmo+ncal*nfilt1 +j)
        ind1.append(ncosmo+ncal*nfilt1 + nsn1+j)
        ind2.append(ncosmo+ncal*nfilt1 +j) 
        ind2.append(ncosmo+ncal*nfilt1 + nsn1+j) 
        ind3.append(ncosmo+ncal*nfilt1 +j)
        ind3.append(ncosmo+ncal*nfilt1 + nsn1+j)   
        
    for i in range(nfilt2):
        ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+2)
        ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+1)
        ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+2)
        ind3.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal)
        ind3.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+1)
        ind3.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+2)
    for j in range(nsn2):
        ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 +j)
        ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + nsn2 +j)
        ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + j) 
        ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + nsn2 + j)   
        ind3.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + j) 
        ind3.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + nsn2 + j)
        
    for i in range(nfilt3):
        ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+2)
        ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+1)
        ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+2)
        ind3.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal)
        ind3.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+1)
        ind3.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+2)
    for j in range(nsn3):
        ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + j)
        ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + nsn3 + j)
        ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + j) 
        ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + nsn3 + j)
        ind3.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + j) 
        ind3.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + nsn3 + j) 
        
    for i in range(nfilt4):
        ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+2)
        ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+1)
        ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+2)
        ind3.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal)
        ind3.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+1)
        ind3.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+2)        
    for j in range(nsn4):
        ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3 + ncal*nfilt4 + j) 
        ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3 + ncal*nfilt4 + nsn4 +j)
        ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3 + ncal*nfilt4 + j) 
        ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3 + ncal*nfilt4 + nsn4 + j)  
        ind3.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3 + ncal*nfilt4 + j) 
        ind3.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3 + ncal*nfilt4 + nsn4 + j)     


    # remove wa
    ind0.append(2)
    ind1.append(2) 
    ind2.append(2)
    ind3.append(2)
    
    _F = numpy.delete(F,ind0,0)
    _F = numpy.delete(_F,ind0,1)
    ans[1,3]= sqrtInvFisherDiag(_F,1)

    
    _F = numpy.delete(F,ind1,0)
    _F = numpy.delete(_F,ind1,1)
    ans[1,2]= sqrtInvFisherDiag(_F,1)
    
    _F = numpy.delete(F,ind2,0)
    _F = numpy.delete(_F,ind2,1)
    ans[1,1]= sqrtInvFisherDiag(_F,1)

    _F = numpy.delete(F,ind3,0)
    _F = numpy.delete(_F,ind3,1)
    ans[1,0]= sqrtInvFisherDiag(_F,1)

    # remove w0
    ind0.append(1)
    ind1.append(1) 
    ind2.append(1)
    ind3.append(1)

    _F = numpy.delete(F,ind0,0)
    _F = numpy.delete(_F,ind0,1)
    ans[0,3]=sqrtInvFisherDiag(_F,0)
    
    _F = numpy.delete(F,ind1,0)
    _F = numpy.delete(_F,ind1,1)
    ans[0,2]= sqrtInvFisherDiag(_F,0)
    
    _F = numpy.delete(F,ind2,0)
    _F = numpy.delete(_F,ind2,1)
    ans[0,1]=sqrtInvFisherDiag(_F,0)
    
    _F = numpy.delete(F,ind3,0)
    _F = numpy.delete(_F,ind3,1)
    ans[0,0]=sqrtInvFisherDiag(_F,0)    

    return ans

# Pantheon Dataset

In [20]:
data = Table.read('../data/apjac8e04t7_mrt.txt', format="ascii.cds")
df = data.to_pandas()
# print(df['CID'].unique())

In [21]:
df = df[["CID", "zHD", "e_mBcorr","e_mB", "c"]]
df = df.groupby(['CID']).mean()  # not quite right for error propagation
df =df.sort_values(by='zHD')

In [22]:
## Single Band

In [23]:
cosmo = jc.Planck15()
Omega_c=cosmo.Omega_c
w0=-0.9
wa=cosmo.wa

snmodel = SNModel()

dm_int =0.14
ncal=3
ncosmo=3
R=4.5
efflams_all = numpy.array([4700., 6420.,  7849., 8500, 12500, 16000 ])

# Consider 4 Surveys
# z<0.15
#dfs = df.loc[(df['zHD'] > 0.01) & (df['zHD'] < 0.15)]  # to run faster
dfs = df.loc[(df['zHD'] < 0.15)]  # to run faster
efflams=efflams_all[0:1]

zs = dfs['zHD'].values
Avs = dfs['c'].values
Rvs = numpy.zeros(len(zs))+2.8
dm_data =  dfs['e_mBcorr'].values

nsn1 = len(zs)
nfilt1 = len(efflams)
dCs= numpy.zeros((nfilt1,ncal))

F =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data*numpy.sqrt(nfilt1), snmodel, R)
# addCalibrationUncertainty(ncosmo, nfilt1, ncal, F)

# # 0.15 < z < 0.55
dfs = df.loc[(df['zHD'] > 0.15) & (df['zHD'] < 0.55)]
efflams=efflams_all[1:2]

zs = dfs['zHD'].values
Avs = dfs['c'].values
Rvs = numpy.zeros(len(zs))+2.8

dm_data =  dfs['e_mBcorr'].values

nsn2 = len(zs)
nfilt2 = len(efflams)
dCs= numpy.zeros((nfilt2,ncal))

F_ =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data*numpy.sqrt(nfilt2), snmodel, R)
# addCalibrationUncertainty(ncosmo, nfilt2, ncal, F_)

F = F_merge(ncosmo, F, F_)

# # # 0.55 < z < 0.9
dfs = df.loc[(df['zHD'] > 0.55) & (df['zHD'] < 0.9)]
efflams=efflams_all[2:3]
zs = dfs['zHD'].values
Avs = dfs['c'].values
Rvs = numpy.zeros(len(zs))+2.8

dm_data =  dfs['e_mBcorr'].values

nsn3 = len(zs)
nfilt3 = len(efflams)
dCs= numpy.zeros((nfilt3,ncal))

F_ =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data*numpy.sqrt(nfilt3), snmodel, R)
# addCalibrationUncertainty(ncosmo, nfilt3, ncal, F_)

F = F_merge(ncosmo, F, F_)

# # # 0.9 < z
dfs = df.loc[(df['zHD'] > 0.9)]
efflams=efflams_all[3:4]
zs = dfs['zHD'].values
Avs = dfs['c'].values
Rvs = numpy.zeros(len(zs))+2.8

dm_data =  dfs['e_mBcorr'].values

nsn4 = len(zs)
nfilt4 = len(efflams)
dCs= numpy.zeros((nfilt4,ncal))

F_ =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data*numpy.sqrt(nfilt4), snmodel, R)
# addCalibrationUncertainty(ncosmo, nfilt4, ncal, F_)

F = F_merge(ncosmo, F, F_)

I0000 00:00:1710618015.990698       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


In [24]:
file = open('baseline_oneband_test.pkl', 'wb')
pickle.dump([F, ncosmo, ncal, nfilt1, nsn1, nfilt2, nsn2, nfilt3, nsn3, nfilt4, nsn4] , file)
file.close()

In [25]:
file = open('baseline_oneband_test.pkl', 'rb')
F, ncosmo, ncal, nfilt1, nsn1, nfilt2, nsn2, nfilt3, nsn3, nfilt4, nsn4 =pickle.load(file)
file.close()
F_hold=numpy.array(F)
addCalibrationUncertainty(ncosmo, [nfilt1,nfilt2,nfilt3,nfilt4], [ncal,ncal,ncal,ncal], [nsn1,nsn2,nsn3,nsn4], F_hold, ncalinvsig2 = [1/0.015/0.015, 1./64, 1/0.015/0.015])
ans = Ftosigma(F, ncosmo, ncal, nfilt1, nsn1, nfilt2, nsn2, nfilt3, nsn3, nfilt4, nsn4)
print(ans)

[[0.0157642  0.02165544 0.02419911 0.02963217]
 [0.17174086 0.22795326 0.25588068 0.29687801]]


## Three band

In [20]:
cosmo = jc.Planck15()
Omega_c=cosmo.Omega_c
w0=cosmo.w0
wa=cosmo.wa

snmodel = SNModel()

dm_int =0.14
ncal=3
ncosmo=3
R=4.5
efflams_all = numpy.array([4700., 6420.,  7849., 8500, 12500, 16000 ])

# Consider 4 Surveys
# z<0.15
dfs = df.loc[(df['zHD'] > 0.01) & (df['zHD'] < 0.15)]  # to run faster
# dfs = df.loc[(df['zHD'] < 0.15)]  # to run faster
efflams=efflams_all[0:3]

zs = dfs['zHD'].values
Avs = dfs['c'].values
Rvs = numpy.zeros(len(zs))+2.8
dCs= numpy.zeros((3,ncal))
dm_data =  dfs['e_mB'].values

nsn1 = len(zs)
nfilt1 = len(efflams)

F =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data*numpy.sqrt(nfilt1), snmodel, R)
addCalibrationUncertainty(ncosmo, nfilt1, ncal, F)

# # 0.15 < z < 0.55
dfs = df.loc[(df['zHD'] > 0.15) & (df['zHD'] < 0.55)]
efflams=efflams_all[1:4]

zs = dfs['zHD'].values
Avs = dfs['c'].values
Rvs = numpy.zeros(len(zs))+2.8
dCs= numpy.zeros((3,ncal))
dm_data =  dfs['e_mB'].values

nsn2 = len(zs)
nfilt2 = len(efflams)

F_ =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data*numpy.sqrt(nfilt2), snmodel, R)
addCalibrationUncertainty(ncosmo, nfilt2, ncal, F_)

F = F_merge(ncosmo, F, F_)

# # # 0.55 < z < 0.9
dfs = df.loc[(df['zHD'] > 0.55) & (df['zHD'] < 0.9)]
efflams=efflams_all[2:5]
zs = dfs['zHD'].values
Avs = dfs['c'].values
Rvs = numpy.zeros(len(zs))+2.8
dCs= numpy.zeros((3,ncal))
dm_data =  dfs['e_mB'].values

nsn3 = len(zs)
nfilt3 = len(efflams)

F_ =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data*numpy.sqrt(nfilt3), snmodel, R)
addCalibrationUncertainty(ncosmo, nfilt3, ncal, F_)

F = F_merge(ncosmo, F, F_)

# # # 0.9 < z
dfs = df.loc[(df['zHD'] > 0.9)]
efflams=efflams_all[3:6]
zs = dfs['zHD'].values
Avs = dfs['c'].values
Rvs = numpy.zeros(len(zs))+2.8
dCs= numpy.zeros((3,ncal))
dm_data =  dfs['e_mB'].values

nsn4 = len(zs)
nfilt4 = len(efflams)

F_ =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data*numpy.sqrt(nfilt4), snmodel, R)
addCalibrationUncertainty(ncosmo, nfilt4, ncal, F_)

F = F_merge(ncosmo, F, F_)

# print(nsn1,nsn2,nsn3,nsn4)

I0000 00:00:1710455121.149441       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


In [21]:
file = open('baseline_withminz.pkl', 'wb')
pickle.dump([F, ncosmo, ncal, nfilt1, nsn1, nfilt2, nsn2, nfilt3, nsn3, nfilt4, nsn4] , file)
file.close()

### Look at results

In [23]:
file = open('baseline_withminz.pkl', 'rb')
F, ncosmo, ncal, nfilt1, nsn1, nfilt2, nsn2, nfilt3, nsn3, nfilt4, nsn4 =pickle.load(file)
file.close()
F_hold=numpy.array(F)

In [21]:
# F = numpy.array(F_hold)

# indeces to delete
print("LCDM")
ind0=[1,2]
ind1=[1,2]  #REMOVE wa
ind2=[1,2]
for i in range(nfilt1):
    ind1.append(ncosmo+i*ncal+2)
    ind2.append(ncosmo+i*ncal)
    ind2.append(ncosmo+i*ncal+2)

for i in range(nfilt2):
    ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+2)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+2)

for i in range(nfilt3):
    ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+2)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+2)

for i in range(nfilt4):
    ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+2)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+2)

_F = numpy.delete(F,ind0,0)
_F = numpy.delete(_F,ind0,1)
print(sqrtInvFisherDiag(_F,0))
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[0,0]))

_F = numpy.delete(F,ind1,0)
_F = numpy.delete(_F,ind1,1)
print(sqrtInvFisherDiag(_F,0))
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[0,0]))

_F = numpy.delete(F,ind2,0)
_F = numpy.delete(_F,ind2,1)
print(sqrtInvFisherDiag(_F,0))
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[0,0]))


# indeces to delete
print("w0CDM")
ind0=[2]
ind1=[2]  #REMOVE wa
ind2=[2]
for i in range(nfilt1):
    ind1.append(ncosmo+i*ncal+2)
    ind2.append(ncosmo+i*ncal)
    ind2.append(ncosmo+i*ncal+2)

for i in range(nfilt2):
    ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+2)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+2)

for i in range(nfilt3):
    ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+2)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+2)

for i in range(nfilt4):
    ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+2)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+2)

_F = numpy.delete(F,ind0,0)
_F = numpy.delete(_F,ind0,1)
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[1,1]))
print(sqrtInvFisherDiag(_F,1))
# print(numpy.linalg.det(Finv[0:2,0:2]))

_F = numpy.delete(F,ind1,0)
_F = numpy.delete(_F,ind1,1)
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[1,1]))
print(sqrtInvFisherDiag(_F,1))
# print(numpy.linalg.det(Finv[0:2,0:2]))

_F = numpy.delete(F,ind2,0)
_F = numpy.delete(_F,ind2,1)
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[1,1]))
print(sqrtInvFisherDiag(_F,1))
# print(numpy.linalg.det(Finv[0:2,0:2]))

ind1=[2]  #REMOVE wa and R
ind2=[2]

for i in range(nfilt1):
    ind1.append(ncosmo+i*ncal+2)
    ind2.append(ncosmo+i*ncal)
    ind2.append(ncosmo+i*ncal+2)
for j in range(nsn1):
    ind1.append(ncosmo+ncal*nfilt1 + nsn1+j)
    ind2.append(ncosmo+ncal*nfilt1 + nsn1+j)   
        
for i in range(nfilt2):
    ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+2)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+2)
for j in range(nsn2):
    ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + nsn2 +j)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + nsn2 + j)   

for i in range(nfilt3):
    ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+2)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+2)
for j in range(nsn3):
    ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + nsn3 + j)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + nsn3 + j)  

for i in range(nfilt4):
    ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+2)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+2)
for j in range(nsn4):
    ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3 + ncal*nfilt4 + nsn4 +j)
    ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3 + ncal*nfilt4 + nsn4 + j)      

 

_F = numpy.delete(F,ind1,0)
_F = numpy.delete(_F,ind1,1)
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[1,1]))
print(sqrtInvFisherDiag(_F,1))
# print(numpy.linalg.det(Finv[0:2,0:2]))

_F = numpy.delete(F,ind2,0)
_F = numpy.delete(_F,ind2,1)
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[1,1]))
print(sqrtInvFisherDiag(_F,1))


# print(numpy.linalg.det(Finv[0:2,0:2]))

# # Omega_M prior
# F[0,0]  += 1/0.03**2 # Omega_M prior
# # indeces to delete
# Finv= numpy.linalg.inv(F)
# print(Finv[0:3,0:3])
# # print(numpy.linalg.det(Finv[0:2,0:2]))

# ind1=[]  #REMOVE wa
# ind2=[]
# for i in range(nfilt):
#     ind1.append(ncosmo+i*nfilt+2)
#     ind1.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+2)
#     ind2.append(ncosmo+i*nfilt)
#     ind2.append(ncosmo+i*nfilt+2)
#     ind2.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt)
#     ind2.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+2)
    
# _F = numpy.delete(F,ind1,0)
# _F = numpy.delete(_F,ind1,1)
# Finv= numpy.linalg.inv(_F)
# print(Finv[0:3,0:3])
# # print(numpy.linalg.det(Finv[0:2,0:2]))

# _F = numpy.delete(F,ind2,0)
# _F = numpy.delete(_F,ind2,1)
# Finv= numpy.linalg.inv(_F)
# print(Finv[0:3,0:3])
# # print(numpy.linalg.det(Finv[0:2,0:2]))

LCDM
0.010231190251630879
0.010151119344850376
0.009675057230968068
w0CDM
0.11627832444331375
0.11508178818773679
0.11102202119501227
0.11505253395734405
0.11096870431022575


In [35]:
F[0,0] #without

3765.2520930509936

In [39]:
F[0,0] #with min z

4001.0128541844892

## Debug lowest rung

In [41]:
F.shape

(3125, 3125)

In [ ]:
cosmo = jc.Planck15()
Omega_c=cosmo.Omega_c
w0=cosmo.w0
wa=cosmo.wa

snmodel = SNModel()

dm_int =0.14
ncal=3
ncosmo=3
R=4.5
efflams_all = numpy.array([4700., 6420.,  7849., 8500, 12500, 16000 ])

# Consider 4 Surveys
# z<0.15
# dfs = df.loc[(df['zHD'] > 0.01) & (df['zHD'] < 0.15)]  # to run faster
# dfs = df.loc[(df['zHD'] < 0.15)]  # to run faster
dfs = df.loc[(df['zHD'] < 0.01)]
efflams=efflams_all[0:3]

zs = dfs['zHD'].values
dm_data =  dfs['e_mBcorr'].values
Avs = dfs['c'].values
Rvs = numpy.zeros(len(zs))+2.8
dCs= numpy.zeros((3,ncal))

nsn1 = len(zs)
nfilt1 = len(efflams)

F =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data*numpy.sqrt(nfilt1), snmodel, R)
print(F[0,0])
# addCalibrationUncertainty(ncosmo, nfilt1, ncal, F)
# print(F[0,0])

dfs = df.loc[(df['zHD'] > 0.01) & (df['zHD'] < 0.15)]
efflams=efflams_all[0:3]

zs = dfs['zHD'].values
dm_data =  dfs['e_mBcorr'].values
Avs = dfs['c'].values
Rvs = numpy.zeros(len(zs))+2.8
dCs= numpy.zeros((3,ncal))

nsn1 = len(zs)
nfilt1 = len(efflams)

F =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data*numpy.sqrt(nfilt1), snmodel, R)
print(F[0,0])

dfs = df.loc[((df['zHD'] < 0.15))]
efflams=efflams_all[0:3]

zs = dfs['zHD'].values
dm_data =  dfs['e_mBcorr'].values
Avs = dfs['c'].values
Rvs = numpy.zeros(len(zs))+2.8
dCs= numpy.zeros((3,ncal))

nsn1 = len(zs)
nfilt1 = len(efflams)

F =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data*numpy.sqrt(nfilt1), snmodel, R)
print(F[0,0])

wef

# # 0.15 < z < 0.55
dfs = df.loc[(df['zHD'] > 0.15) & (df['zHD'] < 0.55)]
efflams=efflams_all[1:4]

zs = dfs['zHD'].values
dmags =  dfs['e_mBcorr'].values
Avs = dfs['c'].values
Rvs = numpy.zeros(len(zs))+2.8
dCs= numpy.zeros((3,ncal))

nsn2 = len(zs)
nfilt2 = len(efflams)

F_ =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data*numpy.sqrt(nfilt2), snmodel, R)
print(F_[0,0])
addCalibrationUncertainty(ncosmo, nfilt2, ncal, F_)
print(F_[0,0])
F = F_merge(ncosmo, F, F_)
print(F[0,0])

# # # 0.55 < z < 0.9
dfs = df.loc[(df['zHD'] > 0.55) & (df['zHD'] < 0.9)]
efflams=efflams_all[2:5]
zs = dfs['zHD'].values
dmags =  dfs['e_mBcorr'].values
Avs = dfs['c'].values
Rvs = numpy.zeros(len(zs))+2.8
dCs= numpy.zeros((3,ncal))

nsn3 = len(zs)
nfilt3 = len(efflams)

F_ =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data*numpy.sqrt(nfilt3), snmodel, R)
print(F_[0,0])
addCalibrationUncertainty(ncosmo, nfilt3, ncal, F_)

# F = F_merge(ncosmo, F, F_)

# # # 0.9 < z
dfs = df.loc[(df['zHD'] > 0.9)]
efflams=efflams_all[3:6]
zs = dfs['zHD'].values
dmags =  dfs['e_mBcorr'].values
Avs = dfs['c'].values
Rvs = numpy.zeros(len(zs))+2.8
dCs= numpy.zeros((3,ncal))

nsn4 = len(zs)
nfilt4 = len(efflams)

F_ =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data*numpy.sqrt(nfilt4), snmodel, R)
print(F_[0,0])
addCalibrationUncertainty(ncosmo, nfilt4, ncal, F_)

# F = F_merge(ncosmo, F, F_)

0.7558492782989573


In [28]:
# F = numpy.array(F_hold)

# indeces to delete
print("LCDM")
ind0=[1,2]
ind1=[1,2]  #REMOVE wa
ind2=[1,2]
for i in range(nfilt1):
    ind1.append(ncosmo+i*ncal+2)
    ind2.append(ncosmo+i*ncal)
    ind2.append(ncosmo+i*ncal+2)

# for i in range(nfilt2):
#     ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+2)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+2)

# for i in range(nfilt3):
#     ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+2)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+2)

# for i in range(nfilt4):
#     ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+2)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+2)

_F = numpy.delete(F,ind0,0)
_F = numpy.delete(_F,ind0,1)
print(sqrtInvFisherDiag(_F,0))
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[0,0]))

_F = numpy.delete(F,ind1,0)
_F = numpy.delete(_F,ind1,1)
print(sqrtInvFisherDiag(_F,0))
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[0,0]))

_F = numpy.delete(F,ind2,0)
_F = numpy.delete(_F,ind2,1)
print(sqrtInvFisherDiag(_F,0))
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[0,0]))


# indeces to delete
print("w0CDM")
ind0=[2]
ind1=[2]  #REMOVE wa
ind2=[2]
for i in range(nfilt1):
    ind1.append(ncosmo+i*ncal+2)
    ind2.append(ncosmo+i*ncal)
    ind2.append(ncosmo+i*ncal+2)

# for i in range(nfilt2):
#     ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+2)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+2)

# for i in range(nfilt3):
#     ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+2)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+2)

# for i in range(nfilt4):
#     ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+2)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+2)

_F = numpy.delete(F,ind0,0)
_F = numpy.delete(_F,ind0,1)
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[1,1]))
print(sqrtInvFisherDiag(_F,1))
# print(numpy.linalg.det(Finv[0:2,0:2]))

_F = numpy.delete(F,ind1,0)
_F = numpy.delete(_F,ind1,1)
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[1,1]))
print(sqrtInvFisherDiag(_F,1))
# print(numpy.linalg.det(Finv[0:2,0:2]))

_F = numpy.delete(F,ind2,0)
_F = numpy.delete(_F,ind2,1)
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[1,1]))
print(sqrtInvFisherDiag(_F,1))
# print(numpy.linalg.det(Finv[0:2,0:2]))

ind1=[2]  #REMOVE wa and R
ind2=[2]

for i in range(nfilt1):
    ind1.append(ncosmo+i*ncal+2)
    ind2.append(ncosmo+i*ncal)
    ind2.append(ncosmo+i*ncal+2)
for j in range(nsn1):
    ind1.append(ncosmo+ncal*nfilt1 + nsn1+j)
    ind2.append(ncosmo+ncal*nfilt1 + nsn1+j)   
        
# for i in range(nfilt2):
#     ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+2)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 +i*ncal+2)
# for j in range(nsn2):
#     ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + nsn2 +j)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + nsn2 + j)   

# for i in range(nfilt3):
#     ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+2)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + i*ncal+2)
# for j in range(nsn3):
#     ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + nsn3 + j)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + nsn3 + j)  

# for i in range(nfilt4):
#     ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+2)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3  + i*ncal+2)
# for j in range(nsn4):
#     ind1.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3 + ncal*nfilt4 + nsn4 +j)
#     ind2.append(ncosmo+ncal*nfilt1 + 2*nsn1 + ncal*nfilt2 + 2*nsn2 + ncal*nfilt3 + 2*nsn3 + ncal*nfilt4 + nsn4 + j)      

 

_F = numpy.delete(F,ind1,0)
_F = numpy.delete(_F,ind1,1)
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[1,1]))
print(sqrtInvFisherDiag(_F,1))
# print(numpy.linalg.det(Finv[0:2,0:2]))

_F = numpy.delete(F,ind2,0)
_F = numpy.delete(_F,ind2,1)
# Finv= numpy.linalg.inv(_F)
# print(numpy.sqrt(Finv[1,1]))
print(sqrtInvFisherDiag(_F,1))


# print(numpy.linalg.det(Finv[0:2,0:2]))

# # Omega_M prior
# F[0,0]  += 1/0.03**2 # Omega_M prior
# # indeces to delete
# Finv= numpy.linalg.inv(F)
# print(Finv[0:3,0:3])
# # print(numpy.linalg.det(Finv[0:2,0:2]))

# ind1=[]  #REMOVE wa
# ind2=[]
# for i in range(nfilt):
#     ind1.append(ncosmo+i*nfilt+2)
#     ind1.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+2)
#     ind2.append(ncosmo+i*nfilt)
#     ind2.append(ncosmo+i*nfilt+2)
#     ind2.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt)
#     ind2.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+2)
    
# _F = numpy.delete(F,ind1,0)
# _F = numpy.delete(_F,ind1,1)
# Finv= numpy.linalg.inv(_F)
# print(Finv[0:3,0:3])
# # print(numpy.linalg.det(Finv[0:2,0:2]))

# _F = numpy.delete(F,ind2,0)
# _F = numpy.delete(_F,ind2,1)
# Finv= numpy.linalg.inv(_F)
# print(Finv[0:3,0:3])
# # print(numpy.linalg.det(Finv[0:2,0:2]))

LCDM
0.11828306737692758
0.11749710709465132
0.10777346996203108
w0CDM
4.593948520076723
4.583789280843563
4.4161805878907785
4.583789280843563
4.4161805878907785


In [25]:
df

,zHD,e_mBcorr,e_mB,c
CID,,,,
2011fe,0.00122,1.5165,0.0340,-0.0705
2012cg,0.00256,0.7905,0.0300,0.1115
1994DRichmond,0.00299,0.8810,0.0320,-0.1120
1981B,0.00317,0.6140,0.0340,-0.0050
2017cbv,0.00331,0.5780,0.0965,-0.1110
...,...,...,...,...
rutledge,1.61505,0.3320,0.0690,0.0640
geta,1.69706,0.3800,0.0880,-0.1640
stone,1.80119,0.2810,0.0800,-0.0240


In [26]:
cosmo = jc.Planck15()
Omega_c=cosmo.Omega_c
w0=cosmo.w0
wa=cosmo.wa
dm_int =0.1
ncal=3
ncosmo=3
efflams = numpy.array([4700., 6420.,  7849.])
ffwhm = numpy.array([1500., 1480.,  1470.])
nfilt=len(efflams)

#survey 1
nsn1=100 # 1300
dm_data =0.15*numpy.sqrt(2)
zs = numpy.linspace(0.3,1,nsn1)
zs = numpy.linspace(0.3**3,1**3,nsn1)
zs = zs**(1/3)

Avs = pAvrvs(size=nsn1)
Rvs = numpy.random.normal(2.8, 0.3,size=nsn1)
dCs= numpy.zeros((3,ncal))

F1 =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data)

# #survey 2
nsn2=100 # 227
dm_data =0.15*numpy.sqrt(2)
zs = numpy.linspace(0.023**3,0.15**3,nsn2)
zs = zs**(1/3)

Avs = pAvrvs(size=nsn2)
Rvs = numpy.random.normal(2.8, 0.3,size=nsn2)
dCs= numpy.zeros((3,ncal))

F2 =  F_all(Omega_c, w0, wa, dCs, Avs, Rvs, zs, efflams, dm_int, dm_data)

F = F_merge(ncosmo, F1,F2)

# print(numpy.linalg.slogdet(F))
# Finv= numpy.linalg.inv(F)
# print(Finv[0:ncosmo,0:ncosmo])


TypeError: F_all() missing 2 required positional arguments: 'snmodel' and 'R'

In [30]:
import pickle
file = open('temp.pkl', 'wb')
pickle.dump(F, file)
file.close()

# Play with result

In [31]:
file = open('temp.pkl', 'rb')
F =pickle.load(file)
file.close()
F_hold=numpy.array(F)

## Calibration error priors and stripping

In [32]:
F = numpy.array(F_hold)

# examine effect if calibration errors
# calibration axes
# by hand put in priors for calibration
ncalinvsig2 = [1./64,1/0.15/0.15, 1/0.15/0.15]
# I checked the ordering of the C variables
for i in range(nfilt):
    for j in range(ncal):
        F[ncosmo+i*nfilt+j,ncosmo+i*nfilt+j] += ncalinvsig2[j]
        F[ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+j,ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+j] += ncalinvsig2[j]


# indeces to delete
ind0=[1,2]
_F = numpy.delete(F,ind0,0)
_F = numpy.delete(_F,ind0,1)
Finv= numpy.linalg.inv(_F)
print(Finv[0,0])

ind1=[1,2]  #REMOVE wa
ind2=[1,2]
for i in range(nfilt):
    ind1.append(ncosmo+i*nfilt+2)
    ind1.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+2)
    ind2.append(ncosmo+i*nfilt)
    ind2.append(ncosmo+i*nfilt+2)
    ind2.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt)
    ind2.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+2)
    
_F = numpy.delete(F,ind1,0)
_F = numpy.delete(_F,ind1,1)
Finv= numpy.linalg.inv(_F)
print(Finv[0,0])

_F = numpy.delete(F,ind2,0)
_F = numpy.delete(_F,ind2,1)
Finv= numpy.linalg.inv(_F)
print(Finv[0,0])


# indeces to delete
ind0=[2]
_F = numpy.delete(F,ind0,0)
_F = numpy.delete(_F,ind0,1)
Finv= numpy.linalg.inv(_F)
print(Finv[0:2,0:2])
# print(numpy.linalg.det(Finv[0:2,0:2]))

ind1=[2]  #REMOVE wa
ind2=[2]
for i in range(nfilt):
    ind1.append(ncosmo+i*nfilt+2)
    ind1.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+2)
    ind2.append(ncosmo+i*nfilt)
    ind2.append(ncosmo+i*nfilt+2)
    ind2.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt)
    ind2.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+2)
    
_F = numpy.delete(F,ind1,0)
_F = numpy.delete(_F,ind1,1)
Finv= numpy.linalg.inv(_F)
print(Finv[0:2,0:2])
# print(numpy.linalg.det(Finv[0:2,0:2]))

_F = numpy.delete(F,ind2,0)
_F = numpy.delete(_F,ind2,1)
Finv= numpy.linalg.inv(_F)
print(Finv[0:2,0:2])
# print(numpy.linalg.det(Finv[0:2,0:2]))

# # Omega_M prior
# F[0,0]  += 1/0.03**2 # Omega_M prior
# # indeces to delete
# Finv= numpy.linalg.inv(F)
# print(Finv[0:3,0:3])
# # print(numpy.linalg.det(Finv[0:2,0:2]))

# ind1=[]  #REMOVE wa
# ind2=[]
# for i in range(nfilt):
#     ind1.append(ncosmo+i*nfilt+2)
#     ind1.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+2)
#     ind2.append(ncosmo+i*nfilt)
#     ind2.append(ncosmo+i*nfilt+2)
#     ind2.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt)
#     ind2.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+2)
    
# _F = numpy.delete(F,ind1,0)
# _F = numpy.delete(_F,ind1,1)
# Finv= numpy.linalg.inv(_F)
# print(Finv[0:3,0:3])
# # print(numpy.linalg.det(Finv[0:2,0:2]))

# _F = numpy.delete(F,ind2,0)
# _F = numpy.delete(_F,ind2,1)
# Finv= numpy.linalg.inv(_F)
# print(Finv[0:3,0:3])
# # print(numpy.linalg.det(Finv[0:2,0:2]))

7.995366490723224e-05
7.732273581370408e-05
7.671439134000466e-05
[[ 0.00488361 -0.01289917]
 [-0.01289917  0.03463791]]
[[ 0.00460984 -0.01219521]
 [-0.01219521  0.0328125 ]]
[[ 0.00458559 -0.01213609]
 [-0.01213609  0.03266551]]


In [ ]:
print(numpy.sqrt(8.060455772927836e-05),numpy.sqrt(0.035))

In [33]:
F = numpy.array(F_hold)

# examine effect if calibration errors
# calibration axes
# by hand put in priors for calibration
ncalinvsig2 = [1./64,1/0.15/0.15, 1/0.15/0.15]
# I checked the ordering of the C variables
for i in range(nfilt):
    for j in range(ncal):
        F[ncosmo+i*nfilt+j,ncosmo+i*nfilt+j] += ncalinvsig2[j]
        F[ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+j,ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+j] += ncalinvsig2[j]

Finv= numpy.linalg.inv(F)

mask=[]
for i in range(3): #nfilt):
    for j in range(ncal):    
        mask.append(ncosmo+i*nfilt+j)
for i in range(3): #nfilt):
    for j in range(ncal):    
        mask.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+j)

print(numpy.sqrt(Finv[numpy.ix_(mask,mask)].diagonal()))


[3.37338016e+00 5.57083839e-03 3.02822944e-03 7.21333481e+00
 5.33863241e-03 9.51546192e-03 7.88118686e+00 5.30194247e-03
 1.99371006e-02 7.89840362e+00 1.25002221e-02 3.70081345e-02
 7.98391911e+00 1.24053701e-02 6.41245871e-02 7.91464934e+00
 1.26313894e-02 2.47461949e-02]


In [ ]:
mask=[0]
for i in range(3): #nfilt):
    for j in range(ncal):    
        mask.append(ncosmo+i*nfilt+j)
for i in range(3): #nfilt):
    for j in range(ncal):    
        mask.append(ncosmo+ncal*nfilt + 2*nsn1 + i*nfilt+j)

print(F[numpy.ix_([0,1],mask)])
Finv= numpy.linalg.inv(F)
FinvCor = Finv / (numpy.outer(numpy.sqrt(Finv.diagonal()),numpy.sqrt(Finv.diagonal()))) 
print(FinvCor[numpy.ix_([0,1],mask)])

In [ ]:
# Omega_M prior
F[0,0]  += 1/0.03**2 # Omega_M prior

## Properties of the DES filter system

In [ ]:
efflams = numpy.array([4700., 6420.,  7849.])
ffwhm = numpy.array([1500., 1480.,  1470.])
nfilt=len(efflams)
# separation between filters in log-lambda
lnlam = numpy.log(efflams)
print(lnlam)
deltalnlam = (lnlam[-1] - lnlam[0])/(nfilt-1)
print(deltalnlam)

# lambda/Delta lambda
print (efflams/ffwhm)

delta = numpy.arcsinh(1/2/4.5)
print(efflams*(numpy.exp(delta)-numpy.exp(-delta)))